In [ ]:
import sqlite3
import pandas as pd
from langchain_community.llms.ollama import Ollama
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
from tqdm import tqdm

from unga79.config import DATA_DIR

DB = DATA_DIR / "countries.db"

In [ ]:
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    rows = cursor.execute("select * from countries;").fetchall()
    column_names = [x[0] for x in cursor.description]
    cursor.close()

df = pd.DataFrame(rows, columns=column_names)
df.head()

In [ ]:
model = Ollama(model="llama3.2")

In [ ]:
template = """You are political analyst. You are helping with speech analysing to find relevant information in the speeches. The following speech was made by an authority of a country in fron the General Assembly of the United Nations. Please answer only based on the content of the speech provided. If you can't find what is asked just reply with "No information provided".

Please answer the following question only based on the content of the speech provided: 
    
    Speech:
    '''{contexto}'''
    
    Question: {input}
    """

system_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_prompt])

chain = chat_prompt | model

In [ ]:
speech_example = df[df["country"] == "Mexico"]["full_speech"].values[0]

In [ ]:
# test
result = chain.invoke(
    {
        "contexto": speech_example,
        # "input": """Please summarize the speech.""",
        # "input": """Identify the countries mentioned in the speech and identify the sentiment as positive, negative, or neutral towards that country. Format: [Country, Sentiment].""",
        # "input": """Summarize the key threats or challenges discussed in the speech by listing the top 3 risks or concerns in bullet points.""",
        "input": """Craft a 3-line haiku inspired by the key themes or messages of the speech.""",
    }
)

print(result)

In [ ]:
update_coutries = """UPDATE countries SET haiku = ? where country = ?"""
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute(update_coutries, (result, "Mexico"))
    conn.commit()
    cursor.close()

In [ ]:
update_coutries = """UPDATE countries SET summary = ? where country = ?"""

pbar = tqdm(df.iterrows(), total=df.shape[0])
for i, r in pbar:
    pbar.set_description(r["country"])
    result = chain.invoke(
        {
            "contexto": r["full_speech"],
            "input": """Please summarize the speech.""",
        }
    )

    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(update_coutries, (result, r["country"]))
        conn.commit()
        cursor.close()

In [ ]:
update_coutries = """UPDATE countries SET countries_mentioned = ? where country = ?"""
pbar = tqdm(df.iterrows(), total=df.shape[0])
for i, r in pbar:
    pbar.set_description(r["country"])
    result = chain.invoke(
        {
            "contexto": r["full_speech"],
            "input": """Identify the countries mentioned in the speech and identify the sentiment as positive, negative, or neutral towards that country. Format: [Country, Sentiment].""",
        }
    )
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(update_coutries, (result, r["country"]))
        conn.commit()
        cursor.close()

In [ ]:
update_coutries = """UPDATE countries SET risks = ? where country = ?"""
pbar = tqdm(df.iterrows(), total=df.shape[0])
for i, r in pbar:
    pbar.set_description(r["country"])
    result = chain.invoke(
        {
            "contexto": r["full_speech"],
            "input": """Summarize the key threats or challenges discussed in the speech by listing the top 3 risks or concerns in bullet points.""",
        }
    )
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(update_coutries, (result, r["country"]))
        conn.commit()
        cursor.close()

In [ ]:
update_coutries = """UPDATE countries SET haiku = ? where country = ?"""
pbar = tqdm(df.iterrows(), total=df.shape[0])
for i, r in pbar:
    pbar.set_description(r["country"])
    result = chain.invoke(
        {
            "contexto": r["full_speech"],
            "input": """Craft a 3-line haiku inspired by the key themes or messages of the speech.""",
        }
    )
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(update_coutries, (result, r["country"]))
        conn.commit()
        cursor.close()